In [2]:
import time
import numpy as np
import pandas as pd
import pickle
import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json
import dateparser

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('headless')    
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service, options=options)

In [88]:
class KommersantParser:
    '''Парсит коммерсант по конкректному региону
    driver - selenium.webdriver, который будете использовать
    day - дата, с какой будем начинать выкачивать новости
    interval: 1 - за день, 2 - новости за неделю, 3 - за месяц с сегодняшней даты
    По умолчанию качает понедельно (interval=week), по региону Урал (region=66), начиная с сегодня'''

    intervals = {'day': '1', 'week': '2', 'month': '3'}

    def __init__(self, driver: webdriver, interval='week', region=66, day=datetime.datetime.today()):
        self.driver = driver
        self.interval = str(interval)
        self.region = str(region)
        self.day = day
    
    
    def __get_page_content(self, api: str) -> BeautifulSoup:
        '''Открывает ссылку, парсит html'''
        while True:
            try:
                self.driver.get(api) # открываем ссылку
                time.sleep(1) # даем прогрузиться
                break
            except TimeoutException:
                time.sleep(2)
                continue
        page_source = self.driver.page_source # берем страницу
        soup = BeautifulSoup(page_source, 'lxml') # парсим

        return soup


    def __get_article(self, link: str) -> str:
        '''Открывает страничку статьи, собирает текст с нее'''

        soup = self.__get_page_content(link)
        # здесь лежит статья
        while True:
            try:
                article = soup.find('div', class_='doc__body article_text_wrapper js-search-mark')
                ps = article.find_all('p', class_='doc__text') # собираем все аттрибуты p
                break
            except AttributeError:
                continue
        # две последние строчки в ps содержат слова подпишитесь и имя автора, они нам не нужны
        text = ''.join([p.text for p in ps[0:len(ps)-2]]).strip() 

        return text
        
    

    def __parse_first_page(self, soup: BeautifulSoup) -> list:
        '''Возвращает список собранных новостей с распрасенной страницы'''

        news = []

        rubrics = soup.find_all('article', class_='uho rubric_lenta__item js-article') # аттрибут новости
        for rubric in rubrics:
            cur_news = {}
            cur_news['description'] = rubric['data-article-description']
            cur_news['news_id'] = rubric['data-article-docsid']
            cur_news['title'] = rubric['data-article-title']
            cur_news['url'] = rubric['data-article-url']
            # дата
            news_date = dateparser.parse(rubric.find('p', class_='uho__tag rubric_lenta__item_tag hide_desktop').text)
            cur_news['published_date'] = news_date
            cur_news['text'] = self.__get_article(cur_news['url'])
            news.append(cur_news)

        return news
        
    
    def open_first_page(self) -> list:
        '''Открывает страницу архива новостей Коммерсанта с заданными вам параметрами из self, 
        возращает список собранных новостей'''

        today_for_starter_link = self.day.strftime('%Y-%m-%d')
        url = f'https://www.kommersant.ru/archive/list/{self.region}/{self.interval}/{today_for_starter_link}'
        soup = self.__get_page_content(url)

        return self.__parse_first_page(soup)

    
    def __parse_lazy_downloads(self, items: json) -> list:
        '''парсит страничку lazydownloads, достает атрибуты, загружает текст каждой новости'''
        parsed_news = []
        
        for item in items:
            cur_news = {}
            cur_news['description'] = item['SubTitle']
            cur_news['news_id'] = item['DocsID']
            cur_news['title'] = item['Title']
            cur_news['url'] = 'https://www.kommersant.ru/doc/' + str(item['DocsID'])
            cur_news['published_date'] = pd.to_datetime(item['DateDoc'], unit='s')
            cur_news['text'] = self.__get_article(cur_news['url'])
            parsed_news.append(cur_news)

        return parsed_news

        
    
    def __lazy_downloads(self, news: list, last_date: str) -> json:
        '''Делает запрос на lazydownloads с последней новости, 
        на которой остановился список выкачанных новостей, либо на последней со стратовой страницы.
        Возвращает обновленный список и отдельно состояние has_next_page, чтобы итерироваться'''

        last_id = news[-1]['news_id']
        url = f'https://www.kommersant.ru/archive/list/lazyloaddocs?regionid={self.region}&date={last_date}&intervaltype={self.intervals[self.interval]}&idafter={last_id}'
        soup = self.__get_page_content(url)
        data = json.loads(soup.text)
        news.extend(self.__parse_lazy_downloads(data['Items']))
        has_next_page = data['HasNextPage']

        return has_next_page

    
    def download_until_date(self, news: list, stop_date=datetime.datetime(2020,1,1)):
        '''Подгружает новости через lazydownloads до тех пор, пока не дойдет до стоп даты'''
        last_date = news[-1]['published_date'].date()
        stop_date = stop_date.date()
        
        while last_date >= stop_date:
            has_next_page = True
            while has_next_page:
                has_next_page = self.__lazy_downloads(news, last_date.strftime('%d.%m.%Y'))
            with open('kommersant', 'wb') as f:
                pickle.dump(news, f)
            print('Updated')
            last_date = news[-1]['published_date'].date()

        print(f'Update a file up to {stop_date.strftime("%d.%m.%Y")}')

In [89]:
parser = KommersantParser(driver=driver, interval='week', region=66)

In [94]:
#news = parser.open_first_page()
parser.download_until_date(news, stop_date=datetime.datetime(2020, 1, 1))

c:\Users\ska19\anaconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [95]:
news

[{'description': '',
  'news_id': '5423686',
  'title': 'В «тюменской матрешке» подтвердили 41 новый случай COVID-19',
  'url': 'https://www.kommersant.ru/doc/5423686',
  'published_date': datetime.datetime(2022, 6, 22, 14, 48),
  'text': 'За прошедшие сутки коронавирус выявили у 41 жителя Тюменской области, Ханты-Мансийского и Ямало-Ненецкого автономных округов, следует из данных сайта «Стопкоронавирус.рф». В Тюменской области диагностировали 25 случаев заражения. За все время пандемии инфекцию выявили у 191 984 жителей региона, из которых 187,4 тыс. – выздоровели, 4 335 – скончались. В ХМАО подтвердили коронавирусную инфекцию у 10 человек. Общее число заболевших составляет 211,1 тыс. Из них 208 389 пациентов уже выздоровели, 2 554 – умерли. На Ямале за сутки заразились шесть человек. С начала пандемии COVID-19 зафиксировали у 137 226 жителей. 135,8 тыс.– выздоровели, 1 042 – скончались.'},
 {'description': '',
  'news_id': '5423672',
  'title': 'У банка «Солидарность» в Екатеринбурге

In [92]:
with open('kommersant', 'wb') as f:
    pickle.dump(news, f)

In [93]:
news[-1]

{'description': '',
 'news_id': 5369059,
 'title': 'Запрет на посещение лесов в Тюменской области продлили до 16 июня',
 'url': 'https://www.kommersant.ru/doc/5369059',
 'published_date': Timestamp('2022-05-26 11:55:46'),
 'text': 'Для жителей Тюменской области до 16 июня продлили запрет на посещение лесов. Соответствующее постановление подписал губернатор Александр Моор, передает информационный центр регионального правительства в telegram-канале. Согласно постановлению, тюменцам запрещено разводить костры, готовить пищу на открытом огне и углях, проводить пожароопасные работы и использовать пиротехнические изделия. Кроме того, в регионе усилили патрулирование лесов.  В случае обнаружения пожара необходимо сообщить о нем по номерам: 8-800-100-94-00, 112.'}